In [1]:
from pathlib import Path
import sys

# More reliable: get the project root from the notebook's location
ROOT_PATH = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
# Or even better for notebooks:
ROOT_PATH = Path().resolve().parent  # Goes up from notebooks/ folder

sys.path.append(str(ROOT_PATH))

In [2]:
import torch
from src.models.ae_res_inception_net import InceptionBlock, AeResInceptionNet

In [3]:
print("=" * 60)
print("Testing AeResInceptionNet")
print("=" * 60)

# Test 1: Basic forward pass
print("\n[Test 1] Basic forward pass")
print("-" * 60)

model = AeResInceptionNet(
    in_channels=1,
    num_classes=7,
    stem_kernel=15,
    stem_downsampling=5,
    stem_channels=32,
    inception_block_num=3,
    inception_kernels=[11, 51, 201],
    inception_bottleneck_channels=16,
    inception_out_channels_per_block=16,
    dropout=0.5,
    fc_hidden_sizes=[128]
)

# Create dummy input: [batch, channels, height, width]
batch_size = 4
height, width = 1000, 64  # Typical time series: many rows, few columns
x = torch.randn(batch_size, 1, height, width)

print(f"Input shape: {x.shape}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Forward pass
model.eval()
with torch.no_grad():
    output = model(x)

print(f"Output shape: {output.shape}")
print(f"Expected output shape: [{batch_size}, 7]")
assert output.shape == (batch_size, 7), f"Output shape mismatch! Got {output.shape}, expected ({batch_size}, 7)"
print("✓ Test 1 passed!")

# Test 2: Check intermediate shapes
print("\n[Test 2] Intermediate feature map shapes")
print("-" * 60)

model.eval()
with torch.no_grad():
    x_test = torch.randn(2, 1, 500, 64)
    
    # Stem
    x_stem = model.conv_stem(x_test)
    print(f"After stem: {x_stem.shape}")
    
    # Through inception blocks
    x_current = x_stem
    for i, block in enumerate(model.inception_blocks):
        x_current = block(x_current)
        print(f"After inception block {i+1}: {x_current.shape}")
    
    # Final output
    output = model.classifier(x_current)
    print(f"Final output: {output.shape}")

print("✓ Test 2 passed!")

# Test 3: Gradient flow
print("\n[Test 3] Gradient flow check")
print("-" * 60)

model.train()
x.requires_grad = True
output = model(x)

# Create a dummy loss
loss = output.mean()
loss.backward()

print(f"Input gradient exists: {x.grad is not None}")
assert x.grad is not None, "Gradients not flowing back to input!"

# Check gradients for some layers
has_grad = any(p.grad is not None for p in model.parameters() if p.requires_grad)
print(f"Model parameters have gradients: {has_grad}")
assert has_grad, "Model parameters don't have gradients!"
print("✓ Test 3 passed!")

# Test 4: Different configurations
print("\n[Test 4] Different configurations")
print("-" * 60)

# Test with different number of inception blocks
for n_blocks in [1, 2, 4]:
    model_test = AeResInceptionNet(
        in_channels=1,
        num_classes=7,
        inception_block_num=n_blocks,
        inception_out_channels_per_block=8,
    )
    x_test = torch.randn(2, 1, 200, 32)
    with torch.no_grad():
        out = model_test(x_test)
    print(f"  {n_blocks} inception blocks: output shape {out.shape} ✓")

# Test with different FC hidden sizes
model_test = AeResInceptionNet(
    in_channels=1,
    num_classes=7,
    fc_hidden_sizes=[256, 128],
)
x_test = torch.randn(2, 1, 200, 32)
with torch.no_grad():
    out = model_test(x_test)
print(f"  Multi-layer FC [256, 128]: output shape {out.shape} ✓")

print("✓ Test 4 passed!")

# Test 5: Output value checks
print("\n[Test 5] Output value checks")
print("-" * 60)

model.eval()
with torch.no_grad():
    output = model(x)

print(f"Output min: {output.min().item():.4f}")
print(f"Output max: {output.max().item():.4f}")
print(f"Output mean: {output.mean().item():.4f}")
print(f"Output std: {output.std().item():.4f}")

# Outputs should be logits (can be any real number)
print("✓ Test 5 passed!")

print("\n" + "=" * 60)
print("All tests passed! ✓")
print("=" * 60)

Testing AeResInceptionNet

[Test 1] Basic forward pass
------------------------------------------------------------
Input shape: torch.Size([4, 1, 1000, 64])
Model parameters: 217,495
Trainable parameters: 217,495
Output shape: torch.Size([4, 7])
Expected output shape: [4, 7]
✓ Test 1 passed!

[Test 2] Intermediate feature map shapes
------------------------------------------------------------
After stem: torch.Size([2, 32, 100, 16])
After inception block 1: torch.Size([2, 64, 100, 16])
After inception block 2: torch.Size([2, 64, 100, 16])
After inception block 3: torch.Size([2, 64, 100, 16])
Final output: torch.Size([2, 7])
✓ Test 2 passed!

[Test 3] Gradient flow check
------------------------------------------------------------
Input gradient exists: True
Model parameters have gradients: True
✓ Test 3 passed!

[Test 4] Different configurations
------------------------------------------------------------
  1 inception blocks: output shape torch.Size([2, 7]) ✓
  2 inception blocks: ou